# MOA-J010321.3-720538

Updating the dataset: F321010200000

* Background model is scaled to match the regions outside of the target aperture.  
* Since there is a star in the LWRS aperture, background is over-estimated, leading to negative fluxes in LiF 1 and both SiC channels.  
* I wrote a script to modify the BKGD limits in all IDF files, re-ran the extraction routine, and combined the resulting spectra.  
* Now the LiF 1 and SiC spectra are fine, but the background is over-estimated for the LiF 2 channel between 980 and 1000 A.  
* The LiF 1 S/N is low, so we use LiF 2 data whenever possible.  
* We scale the FUSE spectrum to match STIS in the region of overlap. 

COMMENT File updated 2 August 2022<br>
COMMENT Star in LWRS aperture caused background to be overestimated.<br>
COMMENT All segments have been scaled to match a STIS spectrum of this star.<br>
COMMENT We do not use the LiF 1 channel.

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E13

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Function to compute offset in pixels between two spectra

def compute_shift(specdata, refdata, wmin, wmax):
    wave_spec  = specdata['wave']
    flux_spec  = specdata['flux']
    error_spec = specdata['error']
    wave_ref   =  refdata['wave']
    flux_ref   =  refdata['flux']
    
    # Compute scale factor between data and reference spectra.
    g = np.where((wave_spec > wmin) & (wave_spec < wmax))
    h = np.where((wave_ref  > wmin) & (wave_ref  < wmax))
    spec_mean = np.mean(flux_spec[g])
    ref_mean  = np.mean(flux_ref[h])
    scale = ref_mean / spec_mean

    # Smooth data and error arrays.
    flux_spec  = convolve(flux_spec,  Box1DKernel(7))
    error_spec = convolve(error_spec, Box1DKernel(7))
    flux_ref   = convolve(flux_ref,   Box1DKernel(7))
    
    # Rescale spectrum to match reference.
    flux_spec *= scale
    error_spec *= scale
    
    # Compute minimum value of chi-squared.
    chisq = np.zeros(20)
    for i in range(20):
        j = i - 10
        chisq[i] = np.sum(((flux_spec[g[0]+j] - flux_ref[h]) / error_spec[g[0]+j])**2)
    
    # We have computed this offset by shifting the flux array, but will 
    # use it by shifting the wavelength array, so must multiply by -1.
    return 10 - np.argmin(chisq)

In [ ]:
# Function to read a regular FUSE spectral file.

def read_fuse(filename):
    f = fits.open(filename)
    data = f[1].data
    f.close()
    spec = make_spectrum (data)
    return data, spec

In [ ]:
# Read header keywords.

filename = 'f321010200000all2ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# The guide channel was LiF 2.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

from specutils.manipulation import (box_smooth)

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Plot the smoothed combined spectra.  
# Note that spectra are scaled to have values near unity.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()

In [ ]:
# All of the channels except LiF 2B have background-subtraction problems.
# A star in the LWRS aperture was included in the background calculation. 
# I have re-defined the background regions and re-extracted all of the spectra.
# Let's look at the results.

lif1a_data, lif1a = read_fuse('F32101020001alif2ttagfcal.fit')
lif1b_data, lif1b = read_fuse('F32101020001blif2ttagfcal.fit')
lif2a_data, lif2a = read_fuse('F32101020002alif2ttagfcal.fit')
lif2b_data, lif2b = read_fuse('F32101020002blif2ttagfcal.fit')
sic1a_data, sic1a = read_fuse('F32101020001asic2ttagfcal.fit')
sic1b_data, sic1b = read_fuse('F32101020001bsic2ttagfcal.fit')
sic2a_data, sic2a = read_fuse('F32101020002asic2ttagfcal.fit')
sic2b_data, sic2b = read_fuse('F32101020002bsic2ttagfcal.fit')

In [ ]:
# Smooth the spectral arrays.

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Plot the smoothed combined spectra.  

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()

# We've solved the background-subtraction problem for LiF 1A and the SiC channels,
# but created a new one for the blue end of LiF 2B.  Something else is going on
# down there; its broad absorption trough is not seen in the other channels.

In [ ]:
# Rescale all four channels to match LiF 2B.

# Select a broad spectral region.

region = SpectralRegion(1040*u.AA, 1070*u.AA)
sub_lif1a = extract_region(lif1a, region)
sub_lif2b = extract_region(lif2b, region)
sub_sic1a = extract_region(sic1a, region)
sub_sic2b = extract_region(sic2b, region)

# Compute ratio of their fluxes to LiF 2B.

mean = sub_lif2b.mean()
scale_lif1a = mean/sub_lif1a.mean()
print ('Scale LiF 1A by', scale_lif1a)

scale_sic1a = mean/sub_sic1a.mean()
print ('Scale SiC 1A by', scale_sic1a)

scale_sic2b = mean/sub_sic2b.mean()
print ('Scale SiC 2B by', scale_sic2b)

In [ ]:
# Scale the smoothed spectra.

lif1a_bsmooth *= scale_lif1a
sic1a_bsmooth *= scale_sic1a
sic2b_bsmooth *= scale_sic2b

In [ ]:
# Plot the rescaled spectra.

f, ax = plt.subplots()  
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()
ax.set_ylim([0,15])
ax.set_xlim([1000,1060])

# The Lyman beta line is broader for LiF 1A than for the other channels.  
# Perhaps it represents the spectrum of a different star.

In [ ]:
# Now consider the long-wavelength spectra.

f, ax = plt.subplots()  
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()

# We scale LiF 1B to match LiF 2A.

In [ ]:
# Select a spectral region.

region = SpectralRegion(1145*u.AA, 1170*u.AA)
sub_lif1b = extract_region(lif1b, region)
sub_lif2a = extract_region(lif2a, region)

# Compute ratio of their fluxes to exp08.

scale_lif1b = sub_lif2a.mean()/sub_lif1b.mean()
print ('Scale LiF 1b by', scale_lif1b)

In [ ]:
# Scale the spectrum.

lif1b_bsmooth *= scale_lif1b

In [ ]:
# Plot the rescaled spectrum.

f, ax = plt.subplots()  
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF1B') 
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF2A') 
ax.legend()
# ax.set_ylim([0,12])

# The absorption features in the LiF 1B spectrum differ from those of LiF 2A.  Let us not use it.

In [ ]:
# From the count-rate plots, we know that even the LiF 2 channel has pointing problems,
# Let's compare with a STIS spectrum of the same star.

filename = 'oe4x08010_x1d.fits'
print ('Target:', fits.getval(filename, 'TARGNAME'))
print ('Aperture', fits.getval(filename, 'APERTURE'))
stis = Spectrum1D.read(filename, format='HST/STIS')
stis_bsmooth = box_smooth(stis, width=15)

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.plot(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A')
ax.legend()
ax.set_xlim([1140,1250])
ax.set_ylim([0,20])

# LiF 2A is fainter than the STIS spectrum.

In [ ]:
# Let's compute the scale factor between the STIS and LiF 2A spectrum.
# Select a spectral region.

region = SpectralRegion(1160*u.AA, 1180*u.AA)
sub_stis = extract_region(stis, region)
sub_lif2a = extract_region(lif2a, region)

# Compute ratio of their fluxes.

scale_to_match_stis = (norm * sub_stis.mean())/sub_lif2a.mean()
print ('Scale by', scale_to_match_stis)

# We will apply this scale factor to the entire spectrum at the very end.

In [ ]:
# Plots the STIS and the rescaled FUSE spectrum.

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.plot(lif2a_bsmooth.spectral_axis, scale_to_match_stis * lif2a_bsmooth.flux, label='LiF 2A')
ax.legend()
ax.set_xlim([1140,1200])
ax.set_ylim([0,20])

# Looks OK.

In [ ]:
# Let's see whether the scale factors derived from LiF 2B work at short wavelengths.

f, ax = plt.subplots() 
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B') 
ax.step(sic1b_bsmooth.spectral_axis, scale_sic1a * sic1b_bsmooth.flux, label='SiC 1B')
ax.step(sic2a_bsmooth.spectral_axis, scale_sic2b * sic2a_bsmooth.flux, label='SiC 2A')
ax.legend()
ax.set_xlim([970, 1010])
ax.set_ylim([-2,15])

# Both LiF 1A and LiF 2B have problems below 1000 A, so we can't us them to scale the SiC channels.
# We'll adopt the scale factors derived from the 1000-1100 A detector segments.
# And we'll use SiC 2A all the way out to 1005 A.

In [ ]:
# Next we compute the shift (in pixels) relative to LiF 2B.

shift_lif1 = compute_shift(lif1a_data, lif2b_data, 1040, 1070)
shift_sic1 = compute_shift(sic1a_data, lif2b_data, 1040, 1070)
shift_sic2 = compute_shift(sic2b_data, lif2b_data, 1040, 1070)

print ('Shift LiF 1 by ', shift_lif1, ' pixels.')
print ('Shift SiC 1 by ', shift_sic1, ' pixels.')
print ('Shift SiC 2 by ', shift_sic2, ' pixels.')

In [ ]:
# Shift all eight spectra.

shift_lif1 *= WPC
shift_sic1 *= WPC
shift_sic2 *= WPC

lif1a_shift = Spectrum1D(spectral_axis=lif1a.spectral_axis + shift_lif1 * u.AA, flux=lif1a.flux)
lif1b_shift = Spectrum1D(spectral_axis=lif1b.spectral_axis + shift_lif1 * u.AA, flux=lif1b.flux)
lif2a_shift = lif2a
lif2b_shift = lif2b
sic1a_shift = Spectrum1D(spectral_axis=sic1a.spectral_axis + shift_sic1 * u.AA, flux=sic1a.flux)
sic1b_shift = Spectrum1D(spectral_axis=sic1b.spectral_axis + shift_sic1 * u.AA, flux=sic1b.flux)
sic2a_shift = Spectrum1D(spectral_axis=sic2a.spectral_axis + shift_sic2 * u.AA, flux=sic2a.flux)
sic2b_shift = Spectrum1D(spectral_axis=sic2b.spectral_axis + shift_sic2 * u.AA, flux=sic2b.flux)


f, ax = plt.subplots()  
ax.step(lif1a_shift.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A')
ax.step(lif2b_shift.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_shift.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_shift.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B') 
ax.legend()
ax.set_xlim([1060,1070])
ax.set_ylim([0,15])

In [ ]:
# We assemble the spectrum, omitting the LiF 1 channel.  Use SiC 1B when possible.

filename = 'f321010200000nvo2ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

data['flux'] = 0.
data['error'] = 0.

# Scale SiC 1B
sic1b_wave = sic1b_shift.spectral_axis.value
h = np.where((sic1b_wave > 899.99) & (sic1b_wave < 980))
g = np.where((data['wave'] > np.min(sic1b_wave[h]) - 0.01) & (data['wave'] < 980))
data['flux'][g] = float(scale_sic1a) * sic1b_data['flux'][h]
data['error'][g] = float(scale_sic1a) * sic1b_data['error'][h]

# Scale SiC 2A
sic2a_wave = sic2a_shift.spectral_axis.value
g = np.where((data['wave'] > 980) & (data['wave'] < 1005))
h = np.where((sic2a_wave > 980) & (sic2a_wave < 1005))
data['flux'][g] = float(scale_sic2b) * sic2a_data['flux'][h]
data['error'][g] = float(scale_sic2b) * sic2a_data['error'][h]

# Scale LiF 2B
lif2b_wave = lif2b_shift.spectral_axis.value
g = np.where((data['wave'] > 1005) & (data['wave'] < 1075))
h = np.where((lif2b_wave > 1005) & (lif2b_wave < 1075))
data['flux'][g] = lif2b_data['flux'][h]
data['error'][g] = lif2b_data['error'][h]

# Scale SiC 1A 
sic1a_wave = sic1a_shift.spectral_axis.value
g = np.where((data['wave'] > 1075) & (data['wave'] < 1087))
h = np.where((sic1a_wave > 1075) & (sic1a_wave < 1087))
data['flux'][g] = float(scale_sic1a) * sic1a_data['flux'][h]
data['error'][g] = float(scale_sic1a) * sic1a_data['error'][h]

# Scale LiF 2A
lif2a_wave = lif2a_shift.spectral_axis.value
g = np.where((data['wave'] > 1087) & (data['wave'] < 1181))
h = np.where((lif2a_wave > 1087) & (lif2a_wave < 1181))
data['flux'][g] = lif2a_data['flux'][h]
data['error'][g] = lif2a_data['error'][h]

# Now scale data and error arrays to match STIS.
data['flux'] *= float(scale_to_match_stis)
data['error'] *= float(scale_to_match_stis)

hdr['comment'] = ''
hdr['comment'] = 'File updated 2 August 2022'
hdr['comment'] = 'This NVO file has been reconstructed from the corresponding ALL file.'
hdr['comment'] = 'All segments have been scaled to match the STIS spectrum of this star.'
hdr['comment'] = 'We do not use the LiF 1 channel.'

f.writeto('level0_f321010200000nvo2ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'f321010200000nvo2ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 
f.close()

filename = 'level0_f321010200000nvo2ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 
f.close()

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 
ax2.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux'], label='OLD FLUX') 
ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])

In [ ]:
# Let's look more closely at the regions sampled only by the SiC channels.

f, ax = plt.subplots()  
ax.step(new['wave'], new['flux'], label='NEW FLUX')
ax.step(old['wave'], old['flux'], label='OLD FLUX') 
ax.legend()
ax.set_xlim([970,1010])

# Seems reasonable.